In [4]:
import split_blocks as sb
import pandas as pd
import geopandas as gpd
import multiprocessing

In [5]:
import numpy as np
from shapely.geometry import Polygon

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
squares = gpd.read_file("gis_data/Square_Boundaries-shp/")

In [5]:
squares_list = squares.loc[:,'SQUARE'].unique().tolist()

In [6]:
square_list_len = len(squares_list)
square_list_len

4420

In [23]:
pool = multiprocessing.Pool(3)

In [24]:
# partial_list = squares_list[:5]

# to do - sort list

In [25]:
result_parallel = pd.concat(pool.map(sb.split_on_poly_by_streetname, squares_list))
pool.close()
pool.join()

In [26]:
result_parallel.head(3)

,group,geometry,SQUARE,SSL,STNAME,SQUARE_PART
0,23,"POLYGON ((-77.00704 38.82496, -77.00716 38.824...",6239,6239S 0037,FORRESTER,1.0
1,20,"POLYGON ((-77.01010 38.82431, -77.01010 38.824...",6239,6239S 0052,GALVESTON,1.0
2,6,"POLYGON ((-77.01155 38.82401, -77.01159 38.823...",6239,6239S 0054,MARTIN LUTHER KING JR,1.0


In [27]:
result_parallel.to_file("output/result_street_dissolve.geojson",driver="GeoJSON")

In [17]:
result = pd.DataFrame()
for idx, shape_id in enumerate(squares_list):
    logging.info(f"working on shape_id: {shape_id} number {idx+1} of {square_list_len}")
    try:
        part_result = split_one_poly_into_parts(shape_id)
        if result.empty:
            result = part_result.copy()
        else:
            result = result.append(part_result)
    except:
        logging.info(f"bad shape -- {shape_id}")
    if idx%100 == 0:
        result.to_file("output/result_address_polysplit.geojson",driver="GeoJSON")

In [18]:
result.to_file("output/result_address_polysplit.geojson",driver="GeoJSON")

In [24]:
result_parallel

,group,geometry,SQUARE,SQUARE_PART
0,0,"POLYGON ((-77.00821 38.82441, -77.00802 38.823...",6239,1
1,3,"POLYGON ((-77.00802 38.82391, -77.00821 38.824...",6239,1
2,2,"POLYGON ((-77.01059 38.82431, -77.01051 38.823...",6239,1
3,1,"POLYGON ((-77.00800 38.82374, -77.00802 38.823...",6239,1
0,3,"POLYGON ((-77.01025 38.82448, -77.01009 38.825...",6239,2
1,0,"POLYGON ((-77.01009 38.82513, -77.01025 38.824...",6239,2
2,1,"POLYGON ((-77.01025 38.82448, -77.01025 38.824...",6239,2
3,2,"POLYGON ((-77.00878 38.82451, -77.00915 38.825...",6239,2
0,3,"POLYGON ((-77.07043 38.91402, -77.07000 38.914...",1303,1
1,1,"POLYGON ((-77.07000 38.91439, -77.07043 38.914...",1303,1


In [6]:
address_polysplit = gpd.read_file("output/result_address_polysplit.geojson")

In [7]:
address_polysplit.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
street_dissolve = gpd.read_file("output/block_polys_w_assessment.geojson")

In [11]:
street_dissolve.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich